In [13]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import gym
from collections import deque
import random
import itertools
from torch import nn
from torch.nn import functional
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import pickle

def save_rewards_data(rewards_data, filename='rewards_data.pkl'):
    """Save the rewards data to a pickle file."""
    with open(filename, 'wb') as file:
        pickle.dump(rewards_data, file)
    print(f'Rewards data saved to {filename}')


def load_rewards_data(filename='rewards_data.pkl'):
    """Load the rewards data from a pickle file."""
    try:
        with open(filename, 'rb') as file:
            rewards_data = pickle.load(file)
        print(f'Rewards data loaded from {filename}')
        return rewards_data
    except FileNotFoundError:
        print(f'Error: The file {filename} does not exist. Please check the file path and try again.')
        return None
    except Exception as e:
        print(f'An error occurred while loading the data: {e}')
        return None

cpu


In [14]:
class tilecoding_Mountain:
  def __init__(self, env, num_tile, bins):
    self.env = env
    self.obs_dim = env.observation_space.shape[0]
    self.bins = bins
    self.num_tile = num_tile
    #bins = tuple([bins]*env.observation_space.shape[0])
    self.numTiles = (self.bins**self.obs_dim) * self.num_tile


  def tiles_grid(self, offset_pos):
    #print(offset_pos)
    min_pos, min_vel  = self.env.observation_space.low
    max_pos, max_vel = self.env.observation_space.high
    bins = self.bins

    pos_grid =  np.linspace(min_pos, max_pos, bins + 1)[1:-1] + offset_pos[0]
    vel_grid = np.linspace(min_vel, max_vel, bins + 1)[1:-1] + offset_pos[1]

    grid = [pos_grid, vel_grid]
    return grid #np.array(grid).flatten()
  
  def create_tilings(self):
    offset_pos = (self.env.observation_space.high - self.env.observation_space.low)/(self.num_tile*self.bins)
    n = self.num_tile // 2
    num = -n*offset_pos
    tiling_specs = [-n*offset_pos]
    for i in range(self.num_tile - 1):
      num += offset_pos
      tiling_specs.append(num)

    
    #tiling_specs = [-offset_pos, tuple([0.0]*self.env.observation_space.shape[0]), offset_pos]
    #print(tiling_specs)

    return [self.tiles_grid(offsets) for offsets in tiling_specs]

  def discretize(self, obs, tile_grid):
    encode = []
    for grids in tile_grid:
      #tile_grid = np.array(tile_grid).reshape(self.obs_dim, -1)
      position_tuple = tuple(int(np.digitize(state, grid)) for state, grid in zip(obs, grids))
      #print(position_tuple)
      array = []
      for i in range(self.obs_dim):
        state = [0]*self.bins
        state[position_tuple[i] - 1] = 1
        array.append(state)
      encode.append(array)
    #print(np.array(encode).flatten())
    return np.array(encode).flatten()#np.array(array_160d).reshape(-1, 4)

In [15]:
class Model(nn.Module):
    def __init__(self, input_features, output_values):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(in_features=input_features, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=32)
        self.fc3 = nn.Linear(in_features=32, out_features=output_values)

    def forward(self, x):
        x = functional.selu(self.fc1(x))
        x = functional.selu(self.fc2(x))
        x = self.fc3(x)
        return x

class DDQN:
    def __init__(self, env, bins, num_tiles, discretizer, use_cuda=True, learning_rate=1e-5, gamma=0.7, memory_len=10000, start_epsilon=1):
        self.device = torch.device("cuda" if use_cuda and torch.cuda.is_available() else "cpu")
        self.env = env
        self.discretizer = discretizer
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.tile_grids = self.discretizer.create_tilings()

        self.epsilon = start_epsilon
        self.min_epsilon = 0.0001
        self.epsilon_decay = 0.9 / 2.5e3
        self.memory_len = memory_len
        self.memory = deque(maxlen=self.memory_len)
        self.criterion = nn.MSELoss()

        input_features = env.observation_space.shape[0]*bins*num_tiles
        output_values = env.action_space.n
        self.policy_net = Model(input_features, output_values).to(self.device)
        self.target_net = Model(input_features, output_values).to(self.device)

        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

    def get_states_tensor(self, sample, states_idx):
        sample_len = len(sample)
        states_tensor = torch.empty((sample_len, self.n_features), dtype=torch.float32, requires_grad=False)

        features_range = range(self.n_features)
        for i in range(sample_len):
            for j in features_range:
                states_tensor[i, j] = sample[i][states_idx][j].item()

        return states_tensor

    def normalize_state(self, state):
        if type(state) == tuple:
            state = state[0]
        state[0] /= 1.8
        state[1] /= 0.14
        

    def state_reward(self, state, env_reward):
        return env_reward - (abs(state[0]) + abs(state[1])) / 2.5

    def get_action(self, state, e=0.001):
        
        if random.random() < e:
            return random.randrange(0, self.env.action_space.n)
        else:
            encoded_state = self.discretizer.discretize(state, self.tile_grids)
            encoded_state = torch.tensor(encoded_state, dtype=torch.float32, device=self.device)
            return self.policy_net(encoded_state).argmax().item()

    def fit(self, model, inputs, labels):
        inputs = inputs.to(self.device)
        labels = labels.to(self.device)
        train_ds = TensorDataset(inputs, labels)
        train_dl = DataLoader(train_ds, batch_size=5)

        optimizer = torch.optim.Adam(params=model.parameters(), lr=self.learning_rate)
        model.train()
        total_loss = 0.0

        for x, y in train_dl:
            out = model(x)
            loss = self.criterion(out, y)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        return total_loss / len(train_dl)

    def optimize_model(self, train_batch_size=100):
        if len(self.memory) < train_batch_size:
            return
        train_sample = random.sample(self.memory, train_batch_size)

        state = torch.tensor([s[0] for s in train_sample], dtype=torch.float32, device=device)
        next_state = torch.tensor([s[3] for s in train_sample], dtype=torch.float32, device=device)

        q_estimates = self.policy_net(state).detach()
        next_state_q_estimates = self.target_net(next_state).detach()
        next_actions = self.policy_net(next_state).argmax(dim=1)

        for i in range(len(train_sample)):
            next_action = next_actions[i].item()
            q_estimates[i][train_sample[i][1]] = (train_sample[i][2] +
                                                  self.gamma * next_state_q_estimates[i][next_action].item())

        self.fit(self.policy_net, state, q_estimates)

    def train_one_episode(self):
        current_state = self.env.reset()
        #self.normalize_state(current_state)
        done = False
        score = 0
        reward = 0
        step = 0
        while not done and step < 200:
            if type(current_state) == tuple:
                current_state = current_state[0]
            action = self.get_action(current_state, self.epsilon)
            next_state, env_reward, done,  _ = self.env.step(action)
            #self.normalize_state(next_state)
            encoded_current_state = self.discretizer.discretize(current_state, self.tile_grids)
            encoded_next_state = self.discretizer.discretize(next_state, self.tile_grids)
            self.memory.append((encoded_current_state, action, env_reward, encoded_next_state))
            current_state = next_state
            score += env_reward
            reward += self.state_reward(next_state, env_reward)

            self.optimize_model(100)
            step += 1

            self.epsilon = max(self.min_epsilon, self.epsilon - self.epsilon_decay)

        return score, reward

    def test(self):
        state = self.env.reset()
        #self.normalize_state(state)
        done = False
        score = 0
        reward = 0
        step = 0
        while not done and step < 200:
            if type(state) == tuple:
                state = state[0]
            action = self.get_action(state, 0)  # Using 0 to bypass random action selection
            state, env_reward, done,  _ = self.env.step(action)
            #self.normalize_state(state)
            score += env_reward
            reward += self.state_reward(state, env_reward)
            step += 1

        return score, reward

In [16]:
env = gym.make('MountainCar-v0')

bins = 15
num_tiles = 2
TC = tilecoding_Mountain(env, num_tiles, bins)


In [17]:
def main(used_alpha, used_epsilon, env, bins, num_tiles, discretizer, target_update_delay, test_delay, memory_len=10000, num_runs=1, num_episode=1000, save_file=""):


    # Initialize storage for reward data
    rewards_data = {epsilon: {alpha: [] for alpha in used_alpha} for epsilon in used_epsilon}

    for epsilon in used_epsilon:
        for alpha in used_alpha:
            print(f"Training with epsilon: {epsilon}, alpha: {alpha}")
            run_rewards = []  # Collect rewards for each run for the current config

            for run in range(num_runs):
                
                # avoid decay
                current_epsilon = epsilon

                print(f"Run {run + 1}/{num_runs}")
                
                # renew the wrapper class
                ddqn = DDQN(
                    env=env, 
                    bins = bins, 
                    num_tiles = num_tiles,
                    discretizer=discretizer, 
                    use_cuda=True, 
                    learning_rate=alpha, 
                    gamma=0.99,
                    memory_len=memory_len, 
                    start_epsilon=current_epsilon
                )

                best_test_reward = 0

                episode_rewards = []  # Collect rewards for each episode in the current run
                for i in range(num_episode):
                    score, reward = ddqn.train_one_episode()

                    print(f'Episode {i + 1}: score: {score} - reward: {reward}')

                    if i % target_update_delay == 0:
                        ddqn.target_net.load_state_dict(ddqn.policy_net.state_dict())
                        ddqn.target_net.eval()

                    if (i + 1) % test_delay == 0:
                        test_score, test_reward = ddqn.test()
                        print(f'Test Episode {i + 1}: test score: {test_score} - test reward: {test_reward}')
                        if test_reward > best_test_reward:
                            print('New best test reward. Saving model')
                            best_test_reward = test_reward
                            # torch.save(ddqn.policy_net.state_dict(), f'policy_net_{run}.pth')

                    # Add the rewards
                    episode_rewards.append(score)

                if num_episode % test_delay != 0:
                    test_score, test_reward = ddqn.test()
                    print(f'Test Episode {num_episode}: test score: {test_score} - test reward: {test_reward}')
                    if test_reward > best_test_reward:
                        print('New best test reward. Saving model')
                        best_test_reward = test_reward
                        # torch.save(ddqn.policy_net.state_dict(), f'policy_net_{run}.pth')

                # save the run rewards
                run_rewards.append(episode_rewards)

                print(f'best test reward: {best_test_reward}')

            # Store the rewards for the current epsilon and alpha configuration
            rewards_data[epsilon][alpha] = run_rewards

    if not(save_file == ""):
        save_rewards_data(rewards_data, save_file)

    return rewards_data

In [18]:
rewards_data = main(
    used_alpha=[1e-4], 
    used_epsilon=[0.001],  # start epsilon
    env=env, 
    bins = bins,
    num_tiles= num_tiles,
    discretizer=TC,
    target_update_delay=2, 
    test_delay=10,
    memory_len=10000, 
    num_runs=1, 
    num_episode=200, 
    save_file="DDQN_Fourier_MC_2.pkl"
)

Training with epsilon: 0.001, alpha: 0.0001
Run 1/3


C:\Users\chery\AppData\Local\Temp\ipykernel_548532\4127737418.py:94: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  state = torch.tensor([s[0] for s in train_sample], dtype=torch.float32, device=device)


Episode 1: score: -200.0 - reward: -240.07220157036008
Episode 2: score: -200.0 - reward: -238.98836813975916
Episode 3: score: -200.0 - reward: -238.45073696667086
Episode 4: score: -200.0 - reward: -245.42027096961303
Episode 5: score: -158.0 - reward: -192.83275748914616
Episode 6: score: -200.0 - reward: -246.32907124360278
Episode 7: score: -200.0 - reward: -239.07326048670888
Episode 8: score: -200.0 - reward: -245.36415101738237
Episode 9: score: -200.0 - reward: -244.80396550163962
Episode 10: score: -200.0 - reward: -246.349596415412
Test Episode 10: test score: -200.0 - test reward: -241.87238678697403
Episode 11: score: -152.0 - reward: -185.84114785885643
Episode 12: score: -162.0 - reward: -200.2527167213796
Episode 13: score: -200.0 - reward: -242.23217981786235
Episode 14: score: -200.0 - reward: -240.6987734166355
Episode 15: score: -164.0 - reward: -198.90780473618875
Episode 16: score: -200.0 - reward: -244.28319812286506
Episode 17: score: -199.0 - reward: -240.63617

In [ ]:
import matplotlib.pyplot as plt

def plot_rewards(rewards_data, algorithm_name="Q learning"):
    """Plot the average performance and IQR of the policy."""
    plt.figure(figsize=(14, 7))

    for epsilon, alphas in rewards_data.items():
        for alpha, data in alphas.items():
            # Calculate average and IQR across runs for each episode
            data = np.array(data)  # Convert to numpy array for easier slicing
            avg_rewards = np.mean(data, axis=0)
            lower_quartile = np.percentile(data, 25, axis=0)
            upper_quartile = np.percentile(data, 75, axis=0)

            episodes = np.arange(len(avg_rewards)) + 1
            plt.plot(episodes, avg_rewards, label=f'ε={epsilon}, α={alpha}')
            plt.fill_between(episodes, lower_quartile, upper_quartile, alpha=0.25)

    plt.title(f'Average Performance and IQR by Epsilon and Alpha ({algorithm_name})')
    plt.xlabel('Episodes')
    plt.ylabel('Average Reward')
    plt.legend()
    plt.show()

In [ ]:
plot_rewards(rewards_data, algorithm_name="DDQN (TC MountainCar)")